### Fitting DCC GARCH model using R - switch kernel

In [22]:
#install.packages("rmgarch")
suppressMessages(library(rmgarch))
suppressMessages(library(parallel))
suppressMessages(library(quantmod))
suppressMessages(library(tidyverse))
suppressMessages(library(xts))
suppressMessages(library(ggplot2))
suppressMessages(library(reshape2))
setwd('~/thesis/')
source("backtesting/fitting_mgarch.R")

In [23]:
etfs <- read.csv("data/return_data.csv", sep=";")

In [24]:
#etfs <- etfs %>% select(-Date)
etfs <- etfs %>% mutate(Date=as.Date(Date, format = "%Y-%m-%d"))

### Converting dataset to xts

In [25]:
xts_etfs <- xts(etfs, order.by = etfs[,1])

In [18]:
# garch models: sGARCH, eGARCH, gjrGARCH
# garch order: 1,1 - 1,0
# distribution model: std

# (sGARCH, 1,0), (sGARCH. 1, 1), (gjrGARCH, 1, 1), (eGARCH, 1, 1)

In [28]:
fit_garch <- function(model_type, garchOrder){
    xspec = ugarchspec(mean.model = list(armaOrder = c(0, 0)), variance.model = list(garchOrder = garchOrder, model = model_type), distribution.model = 'std')
    uspec = multispec(replicate(ncol(xts_etfs)-1, xspec))
    spec1 = dccspec(uspec = uspec, dccOrder = c(1, 1), distribution = 'mvt')
    cl = makePSOCKcluster(4)
    multf = multifit(uspec, xts_etfs[,2:ncol(xts_etfs)], cluster = cl)
    fit1 = dccfit(spec1, data = xts_etfs[,2:ncol(xts_etfs)], fit.control = list(eval.se = TRUE), fit = multf, cluster = cl)           
    stopCluster(cl)
    return(fit1)
}

In [29]:
fit_sGARCH10 = fit_mgarch(1000, "sGARCH", "std", c(1, 0))
#fit_sGARCH10 = fit_garch('sGARCH', c(1,0))
#fit_sGARCH11 = fit_garch('sGARCH', c(1,1))
#fit_gjrGARCH11 = fit_garch('gjrGARCH', c(1,1))
#fit_eGARCH11 = fit_garch('eGARCH', c(1,1))

In [69]:
data.frame(coef(fit_sGARCH10), stringsAsFactors=FALSE) %>% write.csv("fit_sGARCH10.csv")
data.frame(coef(fit_sGARCH11), stringsAsFactors=FALSE) %>% write.csv("fit_sGARCH11.csv")
data.frame(coef(fit_gjrGARCH11), stringsAsFactors=FALSE) %>% write.csv("fit_gjrGARCH11.csv")
data.frame(coef(fit_eGARCH11), stringsAsFactors=FALSE) %>% write.csv("fit_eGARCH11.csv")

In [ ]:
data.frame(lapply(coef(fit1), type.convert), stringsAsFactors=FALSE) %>% write.csv("garch_params.csv")

In [ ]:
last_Omega <- tail(rcov(fit1, output="matrix"), 1)

In [ ]:
tail(rcov(fit1, output="matrix"), 1) %>% write.csv('Omega_t.csv')

In [ ]:
d <- matrix(last_Omega, nrow = 4, byrow = TRUE)

In [ ]:
mat <- matrix(0, nrow = 4, ncol = 4)
mat[upper.tri(mat, diag = TRUE)] <- last_Omega

In [33]:
fit_sGARCH10[1]

[[1]]
    [BZ.F].mu  [BZ.F].omega [BZ.F].alpha1  [BZ.F].shape      [EEM].mu 
  0.016300563   4.192166122   0.467825518   3.034751859   0.062201926 
  [EEM].omega  [EEM].alpha1   [EEM].shape      [EXI].mu   [EXI].omega 
  2.674777768   0.407994588   3.171879789   0.087387059   1.547915899 
 [EXI].alpha1   [EXI].shape     [GC.F].mu  [GC.F].omega [GC.F].alpha1 
  0.636521043   2.930733821   0.042644918   1.408693380   0.112580986 
 [GC.F].shape      [HYG].mu   [HYG].omega  [HYG].alpha1   [HYG].shape 
  3.580777780   0.042461460   0.222774910   0.998999918   2.756593798 
     [IEV].mu   [IEV].omega  [IEV].alpha1   [IEV].shape      [IVV].mu 
  0.073332327   2.072308326   0.576122885   2.890712372   0.089094497 
  [IVV].omega  [IVV].alpha1   [IVV].shape      [IXG].mu   [IXG].omega 
  1.482479819   0.816793970   2.518800842   0.078795840   2.557130177 
 [IXG].alpha1   [IXG].shape      [IXN].mu   [IXN].omega  [IXN].alpha1 
  0.903782524   2.653859206   0.107122174   1.599027791   0.418694082 
  [IXN].shape      [IYR].mu   [IYR].omega  [IYR].alpha1   [IYR].shape 
  2.901917312   0.108004507   1.399410154   0.998999946   2.950245592 
     [TLT].mu   [TLT].omega  [TLT].alpha1   [TLT].shape  [Joint]dcca1 
  0.036940522   0.738370791   0.196019498  10.088666715   0.004066743 
 [Joint]dccb1 [Joint]mshape 
  0.994237940   4.000000374

In [ ]:
#coef(fit1) %>% write.csv('coef.csv')
#install.packages("gmvarkit")
library(gmvarkit)

In [ ]:
etf_forecast <- dccforecast(fit1, n.ahead=1)

In [ ]:
rcov(etf_forecast, output="matrix")

In [ ]:
rcov(etf_forecast, output="matrix")

In [ ]:
etf_forecast

### Using DCC-roll to forecast

In [ ]:
cl = makePSOCKcluster(8)
roll <- dccroll(spec = spec1, data = xts_etfs[,2:11], forecast.length = 1, n.ahead=1, refit.every = 1,
               refit.window = "recursive")

#,             fit.control = list(eval.se = TRUE, stationarity = TRUE, scale = FALSE))
stopCluster(cl)

In [ ]:
data <- as_tibble(rcov(roll, output="matrix")) %>%
        mutate(Date = seq.int(1, nrow(rcov(roll, output="matrix")), 1), .before=EEM) 

print(data)

In [ ]:
data %>% ggplot(aes(x = Date)) +
geom_line(color = Series) +
# Add a second axis and specify its features
labs(
  title="Macroeconomic indicators",
  x = "Year",
  y = "Count")


#rcov(roll, output="matrix")